In [ ]:
import os
import matplotlib.pyplot as plt
from datetime import datetime
import re
from collections import Counter, defaultdict
import emoji
os.getcwd()

In [ ]:
os.chdir('..')

In [ ]:
os.chdir('WhatsApp Chat with xxx') 
# Name redacted for privacy. 
# Replace with the name of the folder for chat for the specific group or person

# Debugging 

In [ ]:
def main():
    file_path = input("Enter the path to your WhatsApp chat export file: ")
    your_name = input("Enter your name as it appears in the chat: ")
    
    content_keywords = input("Enter keywords to filter messages (comma separated): ").split(',')
    
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return

    lines = read_chat(file_path)
    if not lines:
        print("No lines were read from the file. The file might be empty or there was an error reading it.")
        return

    print(f"Read {len(lines)} lines from the file.")
    

    messages_by_content = filter_by_content(lines, content_keywords)
    
    # Print messages filtered by content
    print("\nMessages filtered by content:")
    print_filtered_messages(messages_by_content)

if __name__ == "__main__":
    main()

In [ ]:
def filter_by_content(lines, content_keywords, exact_match=False):
    messages_by_content = []
    
    # Pattern to match date, time, sender, and message
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}),\s(\d{1,2}:\d{2}\s[APM]{2})\s-\s(.+?):\s(.+)'
    
    for line in lines:
        if 'media omitted' in line.lower():  # Skip "media omitted" messages
            continue

        match = re.match(pattern, line)
        if match:
            date_str = match.group(1)  # Date part
            time_str = match.group(2)  # Time part
            sender = match.group(3)    # Sender's name
            message = match.group(4)   # Actual message text
            
            # Check if the message contains any of the specified keywords
            found_match = False
            for keyword in content_keywords:
                if exact_match:
                    if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', message.lower()):
                        found_match = True
                        break
                else:
                    if keyword.lower() in message.lower():
                        found_match = True
                        break
            
            if found_match:
                message_time = f"{date_str} {time_str}"
                messages_by_content.append((sender, message, message_time))
                print(f"Match found: [{message_time}] {sender}: {message}") 
    
    print(f"Total matches found: {len(messages_by_content)}")  
    return messages_by_content

In [ ]:
def read_chat(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.readlines()
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return []
    except Exception as e:
        print(f"Error reading file: {e}")
        return []

def extract_messages(lines):
    messages = defaultdict(list)
    pattern = r'\[(\d{2}/\d{2}/\d{4}, \d{2}:\d{2}:\d{2})\] (.+?): (.+)'
    for line in lines:
        match = re.match(pattern, line)
        if match:
            sender = match.group(2)
            message = match.group(3)
            messages[sender].append(message)
    return messages

def analyze_words(messages):
    words = []
    for message in messages:
        words.extend(re.findall(r'\w+', message.lower()))
    return Counter(words).most_common(10)

def analyze_emojis(messages):
    all_emojis = []
    for message in messages:
        all_emojis.extend([char for char in message if char in emoji.EMOJI_DATA])
    return Counter(all_emojis).most_common(10)

def print_analysis(title, data):
    print(f"\n{title}:")
    for item, count in data:
        print(f"{item}: {count}")

def main():
    file_path = input("Enter the path to your WhatsApp chat export file: ")
    
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return

    lines = read_chat(file_path)
    if not lines:
        print("No lines were read from the file. The file might be empty or there was an error reading it.")
        return

    print(f"Read {len(lines)} lines from the file.")

    messages_by_sender = extract_messages(lines)
    
    if not messages_by_sender:
        print("No messages were extracted. The file might not be in the expected WhatsApp chat export format.")
        print("Example of expected format: [DD/MM/YYYY, HH:MM:SS] Sender Name: Message content")
        return

    print(f"Extracted messages for {len(messages_by_sender)} participants.")

    for sender, messages in messages_by_sender.items():
        print(f"\n--- Analysis for {sender} ---")
        print(f"Number of messages: {len(messages)}")
        print_analysis(f"Top 10 most used words by {sender}", analyze_words(messages))
        print_analysis(f"Top 10 most used emojis by {sender}", analyze_emojis(messages))

if __name__ == "__main__":
    main()

# Without Removing Stopwords 

In [ ]:
def read_chat(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.readlines()
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return []
    except Exception as e:
        print(f"Error reading file: {e}")
        return []

def extract_messages(lines):
    messages = defaultdict(list)
    # Updated pattern to match the format in your file
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}),\s(\d{1,2}:\d{2}\s[APM]{2})\s-\s(.+?):\s(.+)'
    for line in lines:
        match = re.match(pattern, line)
        if match:
            sender = match.group(3)
            message = match.group(4)
            messages[sender].append(message)
    return messages

def analyze_words(messages):
    words = []
    for message in messages:
        words.extend(re.findall(r'\w+', message.lower()))
    return Counter(words).most_common(10)

def analyze_emojis(messages):
    all_emojis = []
    for message in messages:
        all_emojis.extend([char for char in message if char in emoji.EMOJI_DATA])
    return Counter(all_emojis).most_common(10)

def print_analysis(title, data):
    print(f"\n{title}:")
    for item, count in data:
        print(f"{item}: {count}")

def main():
    file_path = input("Enter the path to your WhatsApp chat export file: ")
    
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return

    lines = read_chat(file_path)
    if not lines:
        print("No lines were read from the file. The file might be empty or there was an error reading it.")
        return

    print(f"Read {len(lines)} lines from the file.")

    messages_by_sender = extract_messages(lines)
    
    if not messages_by_sender:
        print("No messages were extracted. The file might not be in the expected WhatsApp chat export format.")
        print("Example of expected format: MM/DD/YY, HH:MM AM/PM - Sender Name: Message content")
        return

    print(f"Extracted messages for {len(messages_by_sender)} participants.")

    for sender, messages in messages_by_sender.items():
        print(f"\n--- Analysis for {sender} ---")
        print(f"Number of messages: {len(messages)}")
        print_analysis(f"Top 10 most used words by {sender}", analyze_words(messages))
        print_analysis(f"Top 10 most used emojis by {sender}", analyze_emojis(messages))

if __name__ == "__main__":
    main()

# Before removing Media Omitted

In [ ]:
def read_chat(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.readlines()
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return []
    except Exception as e:
        print(f"Error reading file: {e}")
        return []

def extract_messages(lines):
    messages = defaultdict(list)
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}),\s(\d{1,2}:\d{2}\s[APM]{2})\s-\s(.+?):\s(.+)'
    for line in lines:
        match = re.match(pattern, line)
        if match:
            sender = match.group(3)
            message = match.group(4)
            messages[sender].append(message)
    return messages

def load_stopwords(stopwords_file):
    try:
        with open(stopwords_file, 'r', encoding='utf-8') as file:
            stopwords = {line.strip().lower() for line in file if line.strip()}
        return stopwords
    except Exception as e:
        print(f"Error reading stopwords file: {e}")
        return set()

def analyze_words(messages, stopwords):
    words = []
    for message in messages:
        words.extend([word for word in re.findall(r'\w+', message.lower()) if word not in stopwords])
    return Counter(words).most_common(10)

def analyze_emojis(messages):
    all_emojis = []
    for message in messages:
        all_emojis.extend([char for char in message if char in emoji.EMOJI_DATA])
    return Counter(all_emojis).most_common(10)

def print_analysis(title, data):
    print(f"\n{title}:")
    for item, count in data:
        print(f"{item}: {count}")

In [ ]:
def main():
    file_path = input("Enter the path to your WhatsApp chat export file: ")
    
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return

    stopwords_file = 'stop_hinglish.txt'  # Path to your stopwords file
    stopwords = load_stopwords(stopwords_file)

    lines = read_chat(file_path)
    if not lines:
        print("No lines were read from the file. The file might be empty or there was an error reading it.")
        return

    print(f"Read {len(lines)} lines from the file.")

    messages_by_sender = extract_messages(lines)
    
    if not messages_by_sender:
        print("No messages were extracted. The file might not be in the expected WhatsApp chat export format.")
        print("Example of expected format: MM/DD/YY, HH:MM AM/PM - Sender Name: Message content")
        return

    print(f"Extracted messages for {len(messages_by_sender)} participants.")

    for sender, messages in messages_by_sender.items():
        print(f"\n--- Analysis for {sender} ---")
        print(f"Number of messages: {len(messages)}")
        print_analysis(f"Top 10 most used words by {sender}", analyze_words(messages,stopwords))
        print_analysis(f"Top 10 most used emojis by {sender}", analyze_emojis(messages))

if __name__ == "__main__":
    main()

In [ ]:
def read_chat(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.readlines()
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return []
    except Exception as e:
        print(f"Error reading file: {e}")
        return []

def extract_messages(lines):
    messages = defaultdict(list)
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}),\s(\d{1,2}:\d{2}\s[APM]{2})\s-\s(.+?):\s(.+)'
    for line in lines:
        match = re.match(pattern, line)
        if match:
            message = match.group(4).strip().lower()
            if "media omitted" not in message:  # Skip messages containing "media omitted"
                sender = match.group(3)
                messages[sender].append(match.group(4))
    return messages

def load_stopwords(stopwords_file):
    try:
        with open(stopwords_file, 'r', encoding='utf-8') as file:
            stopwords = {line.strip().lower() for line in file if line.strip()}
        return stopwords
    except Exception as e:
        print(f"Error reading stopwords file: {e}")
        return set()
    
def analyze_words(messages, stopwords):
    words = []
    for message in messages:
        words.extend([word for word in re.findall(r'\w+', message.lower()) if word not in stopwords])
    return Counter(words).most_common(10)


def count_specific_words(messages, specific_words, stopwords):
    word_counts = defaultdict(int)
    for message in messages:
        words = re.findall(r'\w+', message.lower())
        for word in words:
            if word in specific_words and word not in stopwords:
                word_counts[word] += 1
    return word_counts

def print_word_counts(word_counts):
    print("\nSpecific Word Counts:")
    for word, count in word_counts.items():
        print(f"{word}: {count}")

def print_analysis(title, data):
    print(f"\n{title}:")
    for item, count in data:
        print(f"{item}: {count}")
        
def extract_messages_with_time(lines, your_name):
    sent_messages_by_hour = defaultdict(int)
    received_messages_by_hour = defaultdict(int)
    
    # Pattern to match date, time, sender, and message
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}),\s(\d{1,2}:\d{2}\s[APM]{2})\s-\s(.+?):\s(.+)'
    
    for line in lines:
        if 'media omitted' in line.lower():  # Skip "media omitted" messages
            continue

        match = re.match(pattern, line)
        if match:
            date_str = match.group(1)  # Date part
            time_str = match.group(2)  # Time part
            sender = match.group(3)    # Sender's name
            message_time = f"{date_str} {time_str}"
            
            # Parse the date and time to extract the hour
            try:
                datetime_obj = datetime.strptime(message_time, '%m/%d/%y %I:%M %p')
                hour = datetime_obj.hour

                # Distinguish between sent and received messages
                if sender == your_name:
                    sent_messages_by_hour[hour] += 1
                else:
                    received_messages_by_hour[hour] += 1
            except ValueError:
                continue
    
    return sent_messages_by_hour, received_messages_by_hour

def plot_messages_by_hour(sent_messages_by_hour, received_messages_by_hour):
    hours = list(range(24))  # 0-23 hours
    sent_counts = [sent_messages_by_hour[hour] for hour in hours]
    received_counts = [received_messages_by_hour[hour] for hour in hours]

    plt.figure(figsize=(12, 8))
    
    # Plot sent messages in gray with transparency
    bar1 = plt.bar(hours, sent_counts, color='gray', alpha=0.7, label='Sent Messages', width=0.4, align='center')

    # Plot received messages in red with transparency
    bar2 = plt.bar(hours, received_counts, color='red', alpha=0.7, label='Received Messages', width=0.4, align='edge')

    # Calculate total messages for each hour
    total_messages = [sent_counts[i] + received_counts[i] for i in range(24)]

    # Annotate the bars with raw counts and percentage
    for i, bar in enumerate(bar1):
        total = total_messages[i]
        if total > 150:  # Show annotations only for significant message counts
            percentage = (sent_counts[i] / total) * 100
            plt.text(bar.get_x() + bar.get_width()/2.0, bar.get_height() + 30,
                     f'{sent_counts[i]} ({percentage:.1f}%)', ha='center', va='bottom', fontsize=8)

    for i, bar in enumerate(bar2):
        total = total_messages[i]
        if total > 150:  # Show annotations only for significant message counts
            percentage = (received_counts[i] / total) * 100
            plt.text(bar.get_x() + bar.get_width()/2.0, bar.get_height() + 30,
                     f'{received_counts[i]} ({percentage:.1f}%)', ha='center', va='bottom', fontsize=8)

    # Set titles, labels, and improve visual appearance
    plt.title('Number of Messages Sent and Received by Hour of Day', fontsize=16, pad=20)
    plt.xlabel('Hour of the Day', fontsize=14)
    plt.ylabel('Number of Messages', fontsize=14)
    plt.xticks(hours, [f'{h % 12} {"AM" if h < 12 else "PM"}' for h in hours], rotation=45, fontsize=12)  # Formatting x-axis
    plt.yticks(fontsize=12)

    # Customize the grid and appearance
    plt.grid(True, which='both', axis='y', linestyle='--', linewidth=0.5, color='gray', alpha=0.7)

    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.show()
    
def analyze_emojis(messages):
    all_emojis = []
    for message in messages:
        all_emojis.extend([char for char in message if char in emoji.EMOJI_DATA])
    return Counter(all_emojis).most_common(10)

    
def extract_messages_with_time_and_content(lines, your_name, target_hours, content_keywords):
    messages_by_hour = defaultdict(list)

    # Pattern to match date, time, sender, and message
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}),\s(\d{1,2}:\d{2}\s[APM]{2})\s-\s(.+?):\s(.+)'
    
    for line in lines:
        if 'media omitted' in line.lower():  # Skip "media omitted" messages
            continue

        match = re.match(pattern, line)
        if match:
            date_str = match.group(1)  # Date part
            time_str = match.group(2)  # Time part
            sender = match.group(3)    # Sender's name
            message = match.group(4)   # Actual message text
            
            message_time = f"{date_str} {time_str}"
            
            # Parse the date and time to extract the hour
            try:
                datetime_obj = datetime.strptime(message_time, '%m/%d/%y %I:%M %p')
                hour = datetime_obj.hour
                
                # Check if the hour is in the target hours
                if hour in target_hours:
                    # Check if the message contains any of the specified keywords
                    if any(keyword.lower() in message.lower() for keyword in content_keywords):
                        messages_by_hour[hour].append((sender, message, message_time))
            except ValueError:
                continue
    
    return messages_by_hour

def filter_by_time(lines, target_hours):
    messages_by_hour = defaultdict(list)

    # Pattern to match date, time, sender, and message
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}),\s(\d{1,2}:\d{2}\s[APM]{2})\s-\s(.+?):\s(.+)'
    
    for line in lines:
        if 'media omitted' in line.lower():  # Skip "media omitted" messages
            continue

        match = re.match(pattern, line)
        if match:
            date_str = match.group(1)  # Date part
            time_str = match.group(2)  # Time part
            sender = match.group(3)    # Sender's name
            message = match.group(4)   # Actual message text
            
            message_time = f"{date_str} {time_str}"
            
            # Parse the date and time to extract the hour
            try:
                datetime_obj = datetime.strptime(message_time, '%m/%d/%y %I:%M %p')
                hour = datetime_obj.hour
                
                # Check if the hour is in the target hours
                if hour in target_hours:
                    messages_by_hour[hour].append((sender, message, message_time))
            except ValueError:
                continue
    
    return messages_by_hour

def filter_by_content(lines, content_keywords):
    filtered_messages = []
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}),\s(\d{1,2}:\d{2}\s[APM]{2})\s-\s(.+?):\s(.+)'
    
    for line in lines:
        if 'media omitted' in line.lower():
            continue

        match = re.match(pattern, line)
        if match:
            date_str, time_str, sender, message = match.groups()
            
            if any(keyword.lower() in message.lower() for keyword in content_keywords):
                message_time = f"{date_str} {time_str}"
                filtered_messages.append((sender, message, message_time))
    
    return filtered_messages

def print_filtered_messages(messages):
    if not messages:
        print("No messages found.")
        return
    
    for sender, message, message_time in messages:
        print(f"[{message_time}] {sender}: {message}")
    print("\n" + "="*50 + "\n")

def print_filtered_messages(messages_by_hour):
    for hour, messages in messages_by_hour.items():
        print(f"Messages for {hour % 12} {'AM' if hour < 12 else 'PM'}:")
        for sender, message, message_time in messages:
            print(f"[{message_time}] {sender}: {message}")
        print("\n" + "="*50 + "\n")
        
def extract_messages_by_date(lines, target_date):
    messages = []
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}),\s(\d{1,2}:\d{2}\s[APM]{2})\s-\s(.+?):\s(.+)'
    
    for line in lines:
        match = re.match(pattern, line)
        if match:
            date_str, time_str, sender, message = match.groups()
            
            # Parse the date from the message
            try:
                message_date = datetime.strptime(date_str, '%m/%d/%y').date()
            except ValueError:
                try:
                    message_date = datetime.strptime(date_str, '%d/%m/%y').date()
                except ValueError:
                    print(f"Warning: Couldn't parse date from line: {line}")
                    continue
            
            # Check if the message date matches the target date
            if message_date == target_date:
                messages.append((date_str, time_str, sender, message))
    
    return messages

# TOP-10 Words and Emoji

In [ ]:
def main():
    file_path = input("Enter the path to your WhatsApp chat export file: ")
    
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return

    lines = read_chat(file_path)
    if not lines:
        print("No lines were read from the file. The file might be empty or there was an error reading it.")
        return

    print(f"Read {len(lines)} lines from the file.")

    messages_by_sender = extract_messages(lines)
    
    if not messages_by_sender:
        print("No messages were extracted. The file might not be in the expected WhatsApp chat export format.")
        return

    stopwords_file = 'stop_hinglish.txt'  # Path to your stopwords file
    stopwords = load_stopwords(stopwords_file)

    print(f"Extracted messages for {len(messages_by_sender)} participants.")

    for sender, messages in messages_by_sender.items():
        print(f"\n--- Analysis for {sender} ---")
        print(f"Number of messages: {len(messages)}")
        
        # Top 10 most used words
        print_analysis(f"Top 10 most used words by {sender}", analyze_words(messages, stopwords))
        
        # Top 10 most used emojis
        print_analysis(f"Top 10 most used emojis by {sender}", analyze_emojis(messages))
        
if __name__ == "__main__":
    main()

# Keyword Search

In [ ]:
def main():
    file_path = input("Enter the path to your WhatsApp chat export file: ")
    
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return

    lines = read_chat(file_path)
    if not lines:
        print("No lines were read from the file. The file might be empty or there was an error reading it.")
        return

    print(f"Read {len(lines)} lines from the file.")

    messages_by_sender = extract_messages(lines)
    
    if not messages_by_sender:
        print("No messages were extracted. The file might not be in the expected WhatsApp chat export format.")
        return

    stopwords_file = 'stop_hinglish.txt'  # Path to your stopwords file
    stopwords = load_stopwords(stopwords_file)

    # Get specific words from user input
    specific_words = input("Enter the specific words to search for (comma-separated): ").lower().split(',')
    specific_words = [word.strip() for word in specific_words if word.strip()]

    if not specific_words:
        print("No specific words were provided.")
        return

    print(f"Extracted messages for {len(messages_by_sender)} participants.")

    for sender, messages in messages_by_sender.items():
        print(f"\n--- Analysis for {sender} ---")
        print(f"Number of messages: {len(messages)}")
        # Specific word counts
        word_counts = count_specific_words(messages, specific_words, stopwords)
        print_word_counts(word_counts)

if __name__ == "__main__":
    main()

In [ ]:
def main():
    file_path = input("Enter the path to your WhatsApp chat export file: ")
    
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return
    
    lines = read_chat(file_path)
    if not lines:
        print("No lines were read from the file. The file might be empty or there was an error reading it.")
        return
    print(f"Read {len(lines)} lines from the file.")
    
    content_keywords = input("Enter keywords to filter messages (comma separated): ").split(',')
    content_keywords = [keyword.strip() for keyword in content_keywords]
    
    filtered_messages = filter_by_content(lines, content_keywords)
    
    if filtered_messages:
        print(f"\nFound {len(filtered_messages)} messages matching the keywords:")
        for sender, message, message_time in filtered_messages:
            print(f"[{message_time}] {sender}: {message}")
    else:
        print("\nNo messages found matching the specified keywords.")

if __name__ == "__main__":
    main()

In [ ]:
def main():
    file_path = input("Enter the path to your WhatsApp chat export file: ")
    your_name = input("Enter your name as it appears in the chat: ")
    
    target_hours = list(map(int, input("Enter the target hours (comma separated, 0-23): ").split(',')))
    
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return

    lines = read_chat(file_path)
    if not lines:
        print("No lines were read from the file. The file might be empty or there was an error reading it.")
        return

    print(f"Read {len(lines)} lines from the file.")
    
    # Extract messages for specific hours 
    messages_by_time = filter_by_time(lines, target_hours)
    print(messages_by_time)

    # Print the filtered messages
#     filtered_messages = filter_by_content(messages_by_time, content_keywords)

if __name__ == "__main__":
    main()

In [ ]:
def main():
    file_path = input("Enter the path to your WhatsApp chat export file: ")
    
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return
    
    lines = read_chat(file_path)
    if not lines:
        print("No lines were read from the file. The file might be empty or there was an error reading it.")
        return
    print(f"Read {len(lines)} lines from the file.")
    
    date_str = input("Enter the date to extract messages from (MM/DD/YY): ")
    try:
        target_date = datetime.strptime(date_str, '%m/%d/%y').date()
    except ValueError:
        print("Error: Invalid date format. Please use MM/DD/YY.")
        return
    
    messages = extract_messages_by_date(lines, target_date)
    
    if messages:
        print(f"\nFound {len(messages)} messages for {target_date.strftime('%m/%d/%y')}:")
        for date, time, sender, message in messages:
            print(f"[{date}, {time}] {sender}: {message}")
    else:
        print(f"\nNo messages found for {target_date.strftime('%m/%d/%y')}.")

if __name__ == "__main__":
    main()

# Plots

In [ ]:
def main():
    file_path = input("Enter the path to your WhatsApp chat export file: ")
    your_name = input("Enter your name as it appears in the chat: ")
    
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return

    lines = read_chat(file_path)
    if not lines:
        print("No lines were read from the file. The file might be empty or there was an error reading it.")
        return

    print(f"Read {len(lines)} lines from the file.")

    sent_messages_by_hour, received_messages_by_hour = extract_messages_with_time(lines, your_name)

    # Plot the number of sent and received messages grouped by hour of day
    plot_messages_by_hour(sent_messages_by_hour, received_messages_by_hour)

if __name__ == "__main__":
    main()